# Random Forest

In [207]:
import pandas as pd
import numpy as np
import os 
import seaborn as sns
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
# import graphviz

# Enconders
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [208]:
df = pd.read_csv('..\documents\generated\df_inicial_3%.csv', low_memory=False)
display(df)

,subject_id,charttime_chartevent,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,...,race,admission_location,age,charttime_labevent,value_labevent,valuenum_labevent,valueuom_labevent,priority,admittime,time_since_admission_labevent
0,10036156,2157-07-02 04:00:00,92,92.0,%,O2 Saturation Pulseoxymetry Alarm - Low,Alarms,23.133333,2724,URGENT,...,UNKNOWN,TRANSFER FROM HOSPITAL,88,2157-07-02 04:07:00,12,12.0,mEq/L,ROUTINE,2157-07-01 04:52:00,23.250000
1,10011398,2146-12-16 05:00:00,Sharp,NaN,Unknown,Pain Type,Pain/Sedation,21.750000,2724,SURGICAL SAME DAY ADMISSION,...,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,2146-12-16 02:21:00,3.2,3.2,mg/dL,STAT,2146-12-15 07:15:00,19.100000
2,10018845,2184-10-08 04:38:00,Normal for Race,NaN,Unknown,Skin Color,Skin - Assessment,2.166667,2724,EW EMER.,...,WHITE,EMERGENCY ROOM,91,2184-10-08 20:37:00,NaN,NaN,Unknown,STAT,2184-10-08 02:28:00,18.150000
3,10018081,2133-12-19 12:26:00,Swab,NaN,Unknown,Oral Care,Treatments,19.466667,2724,EW EMER.,...,WHITE,EMERGENCY ROOM,79,2133-12-18 17:28:00,5.1,5.1,mg/dL,ROUTINE,2133-12-18 16:58:00,0.500000
4,10018081,2133-12-19 06:00:00,2 rails up,NaN,Unknown,Side Rails,Restraint/Support Systems,13.033333,2724,EW EMER.,...,WHITE,EMERGENCY ROOM,79,2133-12-18 17:28:00,13.9,13.9,%,ROUTINE,2133-12-18 16:58:00,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358190,10019003,2153-03-28 19:00:00,97,97.0,%,O2 saturation pulseoxymetry,Respiratory,19.583333,Z87891,EW EMER.,...,WHITE,EMERGENCY ROOM,70,2153-03-28 14:04:00,30,30.0,mg/dL,ROUTINE,2153-03-27 23:25:00,14.650000
358191,10023117,2175-07-07 00:00:00,Regular,NaN,Unknown,Breathing pattern/effort,Pulmonary,8.050000,Z87891,OBSERVATION ADMIT,...,WHITE,EMERGENCY ROOM,58,2175-07-06 18:37:00,1.9,1.9,mg/dL,STAT,2175-07-06 15:57:00,2.666667
358192,10023117,2175-03-21 16:00:00,Obeys Commands,6.0,Unknown,GCS - Motor Response,Neurological,16.516667,Z87891,OBSERVATION ADMIT,...,WHITE,TRANSFER FROM HOSPITAL,58,2175-03-21 09:03:00,NaN,NaN,mg/dL,ROUTINE,2175-03-20 23:29:00,9.566667
358193,10023117,2175-03-21 07:17:00,35,35.0,mmHg,Pulmonary Artery Pressure Alarm - High,Alarms,7.800000,Z87891,OBSERVATION ADMIT,...,WHITE,TRANSFER FROM HOSPITAL,58,2175-03-21 09:03:00,NaN,NaN,Unknown,ROUTINE,2175-03-20 23:29:00,9.566667


In [184]:
df = df.drop(columns=['subject_id','charttime_chartevent', 'charttime_labevent', 'admittime'])

In [185]:
display(df)

,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,92,92.0,%,O2 Saturation Pulseoxymetry Alarm - Low,Alarms,23.133333,2724,URGENT,Other,UNKNOWN,TRANSFER FROM HOSPITAL,88,12,12.0,mEq/L,ROUTINE,23.250000
1,Sharp,NaN,Unknown,Pain Type,Pain/Sedation,21.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,3.2,3.2,mg/dL,STAT,19.100000
2,Normal for Race,NaN,Unknown,Skin Color,Skin - Assessment,2.166667,2724,EW EMER.,Other,WHITE,EMERGENCY ROOM,91,NaN,NaN,Unknown,STAT,18.150000
3,Swab,NaN,Unknown,Oral Care,Treatments,19.466667,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79,5.1,5.1,mg/dL,ROUTINE,0.500000
4,2 rails up,NaN,Unknown,Side Rails,Restraint/Support Systems,13.033333,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79,13.9,13.9,%,ROUTINE,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358190,97,97.0,%,O2 saturation pulseoxymetry,Respiratory,19.583333,Z87891,EW EMER.,Other,WHITE,EMERGENCY ROOM,70,30,30.0,mg/dL,ROUTINE,14.650000
358191,Regular,NaN,Unknown,Breathing pattern/effort,Pulmonary,8.050000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,EMERGENCY ROOM,58,1.9,1.9,mg/dL,STAT,2.666667
358192,Obeys Commands,6.0,Unknown,GCS - Motor Response,Neurological,16.516667,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58,NaN,NaN,mg/dL,ROUTINE,9.566667
358193,35,35.0,mmHg,Pulmonary Artery Pressure Alarm - High,Alarms,7.800000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58,NaN,NaN,Unknown,ROUTINE,9.566667


In [186]:
df_sample = df.groupby('icd_code', group_keys=False).sample(frac=0.08, random_state=42)
display(df_sample)

,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
67370,Grade 0,NaN,Unknown,20 G Infiltration Scale,Access Lines - Peripheral,16.350000,2724,DIRECT EMER.,Other,WHITE,CLINIC REFERRAL,67,11.4,11.4,g/dL,STAT,15.783333
42599,Children,NaN,Unknown,Support Systems,Restraint/Support Systems,16.266667,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79,32.9,32.9,pg,ROUTINE,0.500000
62299,Unknown,NaN,Unknown,Restraint Location,Restraint/Support Systems,7.066667,2724,SURGICAL SAME DAY ADMISSION,Medicare,OTHER,PHYSICIAN REFERRAL,66,0,0.0,mEq/L,Unknown,4.000000
10137,24,24.0,insp/min,Respiratory Rate,Respiratory,7.033333,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79,136,136.0,mEq/L,STAT,8.750000
30998,Brisk,NaN,Unknown,Pupil Response Right,Neurological,21.550000,2724,URGENT,Other,WHITE,TRANSFER FROM HOSPITAL,56,110,110.0,IU/L,STAT,11.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344457,Clear,NaN,Unknown,RUL Lung Sounds,Pulmonary,18.200000,Z87891,EW EMER.,Other,WHITE,TRANSFER FROM HOSPITAL,72,7.5,7.5,units,STAT,15.550000
349032,Easily Palpable,NaN,Unknown,Dorsal PedPulse L,Cardiovascular (Pulses),12.600000,Z87891,EW EMER.,Other,WHITE,PHYSICIAN REFERRAL,59,7.2,7.2,g/dL,ROUTINE,22.883333
316548,410,410.0,mL,Tidal Volume (spontaneous),Respiratory,15.433333,Z87891,EW EMER.,Medicare,UNKNOWN,EMERGENCY ROOM,79,NaN,NaN,Unknown,STAT,12.300000
355089,0,0.0,Unknown,18 Gauge placed in the field,Access Lines - Peripheral,1.266667,Z87891,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,89,94,94.0,mEq/L,ROUTINE,10.083333


In [187]:
print(df_sample.columns)

Index(['value_chartevent', 'valuenum_chartevent', 'valueuom_chartevent',
       'label_chartevent', 'category', 'time_since_admission_chartevent',
       'icd_code', 'admission_type', 'insurance', 'race', 'admission_location',
       'age', 'value_labevent', 'valuenum_labevent', 'valueuom_labevent',
       'priority', 'time_since_admission_labevent'],
      dtype='object')


In [188]:
print("original distribution:")
print(df_sample['icd_code'].value_counts(normalize=True).head())

print("\n now distribution:")
print(df_sample['icd_code'].value_counts(normalize=True).head())

original distribution:
icd_code
4019      0.268460
2724      0.215662
E785      0.182719
E039      0.144263
Z87891    0.135853
Name: proportion, dtype: float64

 now distribution:
icd_code
4019      0.268460
2724      0.215662
E785      0.182719
E039      0.144263
Z87891    0.135853
Name: proportion, dtype: float64


In [189]:
df_sample.dtypes

value_chartevent                    object
valuenum_chartevent                float64
valueuom_chartevent                 object
label_chartevent                    object
category                            object
time_since_admission_chartevent    float64
icd_code                            object
admission_type                      object
insurance                           object
race                                object
admission_location                  object
age                                  int64
value_labevent                      object
valuenum_labevent                  float64
valueuom_labevent                   object
priority                            object
time_since_admission_labevent      float64
dtype: object

### Variables Defenition

In [205]:

variables = [
   'value_chartevent', 'label_chartevent', 'valueuom_labevent', 'valueuom_chartevent'
]

lab = 'value_labevent'

variables_cat = ['admission_type', 'insurance', 'priority', 'race', 'admission_location', 'category']

numerical_cols = [
    'valuenum_chartevent', 'valuenum_labevent',
    'time_since_admission_chartevent', 'time_since_admission_labevent', 'age'
]
target = 'icd_code'


In [206]:
X_raw = df_sample[variables + numerical_cols + variables_cat + lab].copy()
y = df_sample[target].copy()

TypeError: can only concatenate list (not "str") to list

### String Features

value_chartevent, valueuom_chartevent, label_chartevent, category, admission_type, insurance, race, admission_location, priority



In [199]:
# First the icd-code is an object

le = LabelEncoder()
y_encoded = le.fit_transform(y)

y = y_encoded

In [200]:
# Treat the string columns

df_sample.select_dtypes(include='object').nunique()


value_chartevent       1659
valueuom_chartevent      30
label_chartevent        778
category                 25
icd_code                  6
admission_type            6
insurance                 3
race                      9
admission_location        7
value_labevent         1052
valueuom_labevent        31
priority                  3
dtype: int64

## One-Hot Encoding

In [201]:

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
one_hot_encoded = encoder.fit_transform(X_raw[variables_cat])

one_hot_df = pd.DataFrame(one_hot_encoded, 
                          columns=encoder.get_feature_names_out(variables_cat),
                          index=X_raw.index)

X_encoded = pd.concat([X_raw.drop(columns=variables_cat), one_hot_df], axis=1)


In [202]:
display(X_encoded)

,value_chartevent,label_chartevent,valueuom_labevent,valueuom_chartevent,valuenum_chartevent,valuenum_labevent,time_since_admission_chartevent,time_since_admission_labevent,age,value_labevent,...,category_Pain/Sedation,category_Pulmonary,category_Respiratory,category_Restraint/Support Systems,category_Routine Vital Signs,category_Skin - Assessment,category_Skin - Impairment,category_Skin - Incisions,category_Toxicology,category_Treatments
67370,Grade 0,20 G Infiltration Scale,g/dL,Unknown,NaN,11.4,16.350000,15.783333,67,11.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42599,Children,Support Systems,pg,Unknown,NaN,32.9,16.266667,0.500000,79,32.9,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
62299,Unknown,Restraint Location,mEq/L,Unknown,NaN,0.0,7.066667,4.000000,66,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10137,24,Respiratory Rate,mEq/L,insp/min,24.0,136.0,7.033333,8.750000,79,136,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30998,Brisk,Pupil Response Right,IU/L,Unknown,NaN,110.0,21.550000,11.483333,56,110,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344457,Clear,RUL Lung Sounds,units,Unknown,NaN,7.5,18.200000,15.550000,72,7.5,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
349032,Easily Palpable,Dorsal PedPulse L,g/dL,Unknown,NaN,7.2,12.600000,22.883333,59,7.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316548,410,Tidal Volume (spontaneous),Unknown,mL,410.0,NaN,15.433333,12.300000,79,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
355089,0,18 Gauge placed in the field,mEq/L,Unknown,0.0,94.0,1.266667,10.083333,89,94,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 'value_chartevent', 'label_chartevent' Treatment -> Frequency Encoding

In [203]:
for col in variables:
    freq = X_encoded[col].value_counts()
    X_encoded[col + '_freq'] = X_encoded[col].map(freq)

# Remover os originais de alta cardinalidade
X_encoded.drop(columns=variables, inplace=True)

for col in numerical_cols:
    X_encoded[col + '_missing'] = X_encoded[col].isna().astype(int)

# Preencher os NaNs com um valor neutro
X_encoded[numerical_cols] = X_encoded[numerical_cols].fillna(-999)

display(X_encoded)

,valuenum_chartevent,valuenum_labevent,time_since_admission_chartevent,time_since_admission_labevent,age,value_labevent,admission_type_DIRECT EMER.,admission_type_ELECTIVE,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,...,category_Treatments,value_chartevent_freq,label_chartevent_freq,valueuom_labevent_freq,valueuom_chartevent_freq,valuenum_chartevent_missing,valuenum_labevent_missing,time_since_admission_chartevent_missing,time_since_admission_labevent_missing,age_missing
67370,-999.0,11.4,16.350000,15.783333,67,11.4,1.0,0.0,0.0,0.0,...,0.0,289,95,1165,20661,1,0,0,0,0
42599,-999.0,32.9,16.266667,0.500000,79,32.9,0.0,0.0,1.0,0.0,...,0.0,18,82,607,20661,1,0,0,0,0
62299,-999.0,0.0,7.066667,4.000000,66,0,0.0,0.0,0.0,0.0,...,0.0,935,69,5386,20661,1,0,0,0,0
10137,24.0,136.0,7.033333,8.750000,79,136,0.0,0.0,1.0,0.0,...,0.0,63,666,5386,978,0,0,0,0,0
30998,-999.0,110.0,21.550000,11.483333,56,110,0.0,0.0,0.0,0.0,...,0.0,208,121,801,20661,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344457,-999.0,7.5,18.200000,15.550000,72,7.5,0.0,0.0,1.0,0.0,...,0.0,370,111,838,20661,1,0,0,0,0
349032,-999.0,7.2,12.600000,22.883333,59,7.2,0.0,0.0,1.0,0.0,...,0.0,307,115,1165,20661,1,0,0,0,0
316548,410.0,-999.0,15.433333,12.300000,79,NaN,0.0,0.0,1.0,0.0,...,0.0,3,10,4235,129,0,1,0,0,0
355089,0.0,94.0,1.266667,10.083333,89,94,0.0,0.0,1.0,0.0,...,0.0,831,57,5386,20661,0,0,0,0,0


In [ ]:
lab = 'value_labevent'

X_encoded[lab] = X_encoded[lab ].astype(str)



TypeError: the first argument must be callable

In [114]:
X_encoded.to_csv("df_treated.csv", index=False)

# Random Forest Regressor Model

In [118]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

X = X_encoded

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [120]:
X_train.select_dtypes(include=['object']).nunique()


value_labevent    1038
dtype: int64

In [119]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

ValueError: could not convert string to float: '___'

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)